In [1]:
import glob
import os
import sys

try:
    sys.path.append(glob.glob('carla/PythonAPI/carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

import argparse
import math
# import gin
# gin.enter_interactive_mode()
import hydra
from omegaconf import DictConfig

import logging
from numpy import random
import time

from utils.weather import Weather
from utils.vehicle_manager import VehicleManager
from utils.common import get_actor_blueprints

c:\Users\aadim\workspace\carla-pgm\utils\vehicle_manager.py:25: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="conf", config_name="vehicle_manager")


In [15]:


# @hydra.main(version_base=None, config_path="conf", config_name="config")
# def main(
#     host:str, port:int, number_of_walkers: int, 
#     filterw: str, generationw: str,  
#     seedw: int ):
def main(cfg: DictConfig):

    # @todo cannot import these directly.
    SpawnActor = carla.command.SpawnActor
    SetAutopilot = carla.command.SetAutopilot
    FutureActor = carla.command.FutureActor

    logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)

    
    walkers_list = []
    all_id = []
    client = carla.Client(cfg.host, cfg.port)
    client.set_timeout(10.0)

    # with hydra.initialize(version_base=None, config_path="conf"):
    vcfg = hydra.compose(config_name="vehicle_manager.yaml")
    print(vcfg)
    tm = VehicleManager(client=client, cfg=vcfg)
    try:
        world = client.get_world()

        
        tm.spawn_vehicles()
        blueprintsWalkers = get_actor_blueprints(world, filterw, generationw)
        # -------------
        # Spawn Walkers
        # -------------
        # some settings
        percentagePedestriansRunning = 0.0      # how many pedestrians will run
        percentagePedestriansCrossing = 0.0     # how many pedestrians will walk through the road
        if cfg.seedw:
            world.set_pedestrians_seed(cfg.seedw)
            random.seed(cfg.seedw)
        # 1. take all the random locations to spawn
        spawn_points = []
        for i in range(cfg.number_of_walkers):
            spawn_point = carla.Transform()
            loc = world.get_random_location_from_navigation()
            if (loc != None):
                spawn_point.location = loc
                spawn_points.append(spawn_point)
        # 2. we spawn the walker object
        batch = []
        walker_speed = []
        for spawn_point in spawn_points:
            walker_bp = random.choice(blueprintsWalkers)
            # set as not invincible
            if walker_bp.has_attribute('is_invincible'):
                walker_bp.set_attribute('is_invincible', 'false')
            # set the max speed
            if walker_bp.has_attribute('speed'):
                if (random.random() > percentagePedestriansRunning):
                    # walking
                    walker_speed.append(walker_bp.get_attribute('speed').recommended_values[1])
                else:
                    # running
                    walker_speed.append(walker_bp.get_attribute('speed').recommended_values[2])
            else:
                print("Walker has no speed")
                walker_speed.append(0.0)
            batch.append(SpawnActor(walker_bp, spawn_point))
        results = client.apply_batch_sync(batch, True)
        walker_speed2 = []
        for i in range(len(results)):
            if results[i].error:
                logging.error(results[i].error)
            else:
                walkers_list.append({"id": results[i].actor_id})
                walker_speed2.append(walker_speed[i])
        walker_speed = walker_speed2
        # 3. we spawn the walker controller
        batch = []
        walker_controller_bp = world.get_blueprint_library().find('controller.ai.walker')
        for i in range(len(walkers_list)):
            batch.append(SpawnActor(walker_controller_bp, carla.Transform(), walkers_list[i]["id"]))
        results = client.apply_batch_sync(batch, True)
        for i in range(len(results)):
            if results[i].error:
                logging.error(results[i].error)
            else:
                walkers_list[i]["con"] = results[i].actor_id
        # 4. we put together the walkers and controllers id to get the objects from their id
        for i in range(len(walkers_list)):
            all_id.append(walkers_list[i]["con"])
            all_id.append(walkers_list[i]["id"])
        all_actors = world.get_actors(all_id)

        # wait for a tick to ensure client receives the last transform of the walkers we have just created
        if cfg.asynch or not cfg.synchronous_master:
            world.wait_for_tick()
        else:
            world.tick()

        # 5. initialize each controller and set target to walk to (list is [controler, actor, controller, actor ...])
        # set how many pedestrians can cross the road
        world.set_pedestrians_cross_factor(percentagePedestriansCrossing)
        for i in range(0, len(all_id), 2):
            # start walker
            all_actors[i].start()
            # set walk to random point
            all_actors[i].go_to_location(world.get_random_location_from_navigation())
            # max speed
            all_actors[i].set_max_speed(float(walker_speed[int(i/2)]))

        # print('spawned %d vehicles and %d walkers, press Ctrl+C to exit.' % (len(vehicles_list), len(walkers_list)))

        # Example of how to use Traffic Manager parameters
        # traffic_manager.global_percentage_speed_difference(-30.0)

        # Dynamic Weather
        speed_factor = 1.0
        update_freq = 0.5 / speed_factor


        weather = Weather(world.get_weather())

        elapsed_time = 0.0

        while True:
            if not cfg.asynch and cfg.synchronous_master:
                world.tick()
            else:
                # world.wait_for_tick()
                timestamp = world.wait_for_tick(seconds=30.0).timestamp
                elapsed_time += timestamp.delta_seconds
                if elapsed_time > update_freq:
                    weather.tick(speed_factor * elapsed_time)

                    # traffic_manager.global_percentage_speed_difference(30.0 - (10 * (weather._storm.rain / 100)))
                    world.set_weather(weather.weather)
                    sys.stdout.write('\r' + str(weather) + 12 * ' ')
                    # sys.stdout.write('\n' + str((30.0 - (10 * (weather._storm.rain / 100)))) + 12 * ' ')
                    # sys.stdout.write('\n' + str(world.get_snapshot().find(vehicles_list[10]).get_velocity()) + 12 * ' ')
                    # sys.stdout.write('\n' + str(world.get_snapshot().find(vehicles_list[10]).get_acceleration()) + 12 * ' ')
                    sys.stdout.flush()
                    elapsed_time = 0.0

    finally:

        tm.destroy()

        # stop walker controllers (list is [controller, actor, controller, actor ...])
        for i in range(0, len(all_id), 2):
            all_actors[i].stop()

        print('\ndestroying %d walkers' % len(walkers_list))
        client.apply_batch([carla.command.DestroyActor(x) for x in all_id])

        time.sleep(0.5)

In [16]:
# gin.parse_config_file('configs/config.gin')

try:
    with hydra.initialize(version_base=None, config_path="conf"):
        cfg = hydra.compose(config_name="config.yaml")
        print(cfg)
        main(cfg)
except KeyboardInterrupt:
    pass
finally:
    print('\ndone.')

{'host': '127.0.0.1', 'port': 2000, 'number_of_walkers': 10, 'filterw': 'walker.pedestrian.*', 'generationw': '2', 'seedw': 0}
{'tm_port': 8000, 'safe': True, 'filterv': 'vehicle.*', 'generationv': 'All', 'asynch': True, 'hybrid': False, 'seed': 13, 'car_lights_on': False, 'hero': False, 'respawn': False, 'no_rendering': False}

done.


TypeError: decorated_main() got an unexpected keyword argument 'client'

In [25]:
speed_factor = 2.0
update_freq = 0.1 / speed_factor

client = carla.Client('127.0.0.1', 2000)
client.set_timeout(2.0)
world = client.get_world()

weather = Weather(world.get_weather())

elapsed_time = 0.0

while True:
    timestamp = world.wait_for_tick(seconds=30.0).timestamp
    elapsed_time += timestamp.delta_seconds
    if elapsed_time > update_freq:
        weather.tick(speed_factor * elapsed_time)
        world.set_weather(weather.weather)
        sys.stdout.write('\r' + str(weather) + 12 * ' ')
        sys.stdout.flush()
        elapsed_time = 0.0

Sun(alt: 46.79, azm: 15.15) Storm(clouds=0%, rain=0%, wind=5%)                 

KeyboardInterrupt: 

In [4]:
spawn_points = world.get_map().get_spawn_points()
number_of_spawn_points = len(spawn_points)
number_of_spawn_points

155

In [7]:
weather = world.get_weather()

In [21]:
weather.precipitation = 50
weather.wetness = 30
world.set_weather(weather)
world.get_weather().precipitation

50.0